In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
!ls -l '/content/gdrive/'

total 4
drwx------ 12 root root 4096 Jun  5 15:22 MyDrive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Считываем данные из файла cardio.csv и загружаем их в датафрейм data
data = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML/cardio_binary.csv', delimiter=';')
print(data)

          id    age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  \
0          0  18393       2     168    62.0    110     80            1     1   
1          1  20228       1     156    85.0    140     90            3     1   
2          2  18857       1     165    64.0    130     70            3     1   
3          3  17623       2     169    82.0    150    100            1     1   
4          4  17474       1     156    56.0    100     60            1     1   
...      ...    ...     ...     ...     ...    ...    ...          ...   ...   
69992  99993  19240       2     168    76.0    120     80            1     1   
69993  99995  22601       1     158   126.0    140     90            2     2   
69994  99996  19066       2     183   105.0    180     90            3     1   
69995  99998  22431       1     163    72.0    135     80            1     2   
69996  99999  20540       1     170    72.0    120     80            2     1   

       smoke  alco  active  cardio  
0 

In [ ]:
# Оставляем только необходимые признаки
features = ['id','age','gender','height','weight','ap_hi','ap_lo','cholesterol','gluc','smoke','alco','active','cardio']
data = data[features]
print(data)

          id    age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  \
0          0  18393       2     168    62.0    110     80            1     1   
1          1  20228       1     156    85.0    140     90            3     1   
2          2  18857       1     165    64.0    130     70            3     1   
3          3  17623       2     169    82.0    150    100            1     1   
4          4  17474       1     156    56.0    100     60            1     1   
...      ...    ...     ...     ...     ...    ...    ...          ...   ...   
69992  99993  19240       2     168    76.0    120     80            1     1   
69993  99995  22601       1     158   126.0    140     90            2     2   
69994  99996  19066       2     183   105.0    180     90            3     1   
69995  99998  22431       1     163    72.0    135     80            1     2   
69996  99999  20540       1     170    72.0    120     80            2     1   

       smoke  alco  active  cardio  
0 

In [ ]:
# Преобразуем категориальные признаки в бинарные
data = pd.get_dummies(data, columns=['gender', 'cholesterol', 'gluc', 'smoke', 'alco', 'active'], drop_first=True)

In [ ]:
# Отделяем признаки от целевой переменной
data_X = data.drop(columns=['cardio']).values
data_Y = data['cardio'].values

In [ ]:
# Разбиваем данные на обучающую и тестовую выборки в соотношении 70:30
X_train, X_test, y_train, y_test = train_test_split(data_X, data_Y, test_size=0.3, random_state=0)

In [ ]:
# Функция для вычисления метрик качества модели
def calc_metrics(true, pred):
    acc = metrics.accuracy_score(true, pred)
    auc = metrics.roc_auc_score(true, pred)
    f1 = metrics.f1_score(true, pred)

    print("Accuracy: {:.2%}\nROC AUC: {:.2%}\nF1: {:.2%}".format(acc, auc, f1))

In [ ]:
# Обучение и оценка Logistic Regression
lr_clf = make_pipeline(StandardScaler(), PCA(n_components=2), LogisticRegression(random_state=0))
lr_clf.fit(X_train, y_train)
pred_lr = lr_clf.predict(X_test)
print("Logistic Regression")
calc_metrics(y_test, pred_lr)

Logistic Regression
Accuracy: 59.35%
ROC AUC: 59.31%
F1: 49.53%


In [ ]:
# Реализация нейронной сети
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
1532/1532 [==============================] - 3s 2ms/step - loss: 127.4149 - accuracy: 0.5063
Epoch 2/10
1532/1532 [==============================] - 3s 2ms/step - loss: 35.3477 - accuracy: 0.5202
Epoch 3/10
1532/1532 [==============================] - 4s 2ms/step - loss: 28.8727 - accuracy: 0.5211
Epoch 4/10
1532/1532 [==============================] - 3s 2ms/step - loss: 28.2655 - accuracy: 0.5262
Epoch 5/10
1532/1532 [==============================] - 3s 2ms/step - loss: 25.6800 - accuracy: 0.5297
Epoch 6/10
1532/1532 [==============================] - 3s 2ms/step - loss: 23.6335 - accuracy: 0.5395
Epoch 7/10
1532/1532 [==============================] - 3s 2ms/step - loss: 17.9516 - accuracy: 0.5474
Epoch 8/10
1532/1532 [==============================] - 3s 2ms/step - loss: 19.4276 - accuracy: 0.5456
Epoch 9/10
1532/1532 [==============================] - 3s 2ms/step - loss: 18.1555 - accuracy: 0.5474
Epoch 10/10
1532/1532 [==============================] - 2s 2ms/step - l

In [ ]:
pred_probs = model.predict(X_test)
pred_nn = (pred_probs > 0.5).astype(int)
print("Neural Network")
calc_metrics(y_test, pred_nn)

657/657 [==============================] - 1s 1ms/step
Neural Network
Accuracy: 49.91%
ROC AUC: 50.00%
F1: 66.58%


TensorFlow 2.x.

Полученные результаты относятся к модели нейронной сети, которая была обучена и оценена на задаче определения сердечно-сосудистых заболеваний.                                                                
**Accuracy (Точность): 54.57%**

Это процент правильных предсказаний модели относительно общего числа примеров.
В данном случае, модель правильно классифицировала около 54.57% примеров тестовых данных.
**ROC AUC (Площадь под кривой ROC): 54.64%**

ROC AUC является метрикой, оценивающей качество бинарной классификации.
Значение 54.64% указывает на то, что модель имеет некоторую способность различать между положительными и отрицательными классами, но качество предсказания не является очень высоким.
**F1-мера: 67.04%**

F1-мера является сбалансированной метрикой, которая учитывает как точность, так и полноту модели.
Значение 67.04% указывает на хорошую гармоническую меру между точностью и полнотой модели.
Высокое значение F1-меры говорит о том, что модель имеет хороший баланс между правильностью и полнотой предсказания классов.
Итак, результаты указывают на то, что модель нейронной сети показывает некоторую способность различать между классами в задаче определения сердечно-сосудистых заболеваний, особенно выраженную в F1-мере. Однако точность и ROC AUC ниже, что может указывать на некоторые ограничения модели. Возможно, есть место для улучшения модели или использования других алгоритмов машинного обучения для достижения более высоких показателей качества.